In [1]:
%reload_ext autoreload
%autoreload 2
import os

import ipdb
import matplotlib
from tqdm import tqdm

from torch import nn

from utils.config import opt
from data.dataset import Dataset, TestDataset, inverse_normalize
from data.Data import COCODataset,param_map
from model import FasterRCNNVGG16, FasterRCNNVGG16Extractor
from faster_rcnn_lstm import FasterRCNN_LSTM
from torch.utils import data as data_
from trainer import FasterRCNNTrainer
from utils import array_tool as at
from utils.vis_tool import visdom_bbox
from utils.eval_tool import eval_detection_voc
from model.predictor import PredictNet

C:\Users\Administrator\Desktop\rcnn lstm\model\utils\nms\non_maximum_suppression.py:12: UserWarning: 
    the python code for non_maximum_suppression is about 2x slow
    It is strongly recommended to build cython code: 
    `cd model/utils/nms/; python3 build.py build_ext --inplace
  `cd model/utils/nms/; python3 build.py build_ext --inplace''')


In [2]:
embedding_file = 'G:\\data\\coco_filtered_word2vec_1024'
dataset = COCODataset(embedding_file,opt.n)
dataloader = data_.DataLoader(dataset, \
                              batch_size=1, \
                              shuffle=True, \
                              # pin_memory=True,
                              num_workers=opt.num_workers)
faster_rcnn = FasterRCNNVGG16()
trainer = FasterRCNNTrainer(faster_rcnn).cuda()
trainer.load('G:\\models\\fasterrcnn_torchvision_pretrain.pth')
opt.caffe_pretrain = False
faster_extractor = FasterRCNNVGG16Extractor(trainer.faster_rcnn, opt.n_class, hidden_size=opt.hidden_size).cuda()
lstm = nn.LSTM(input_size=dataset.word_embedding.vector_size, hidden_size=opt.hidden_size, batch_first=True)
predict_param = [dataset.word_embedding.vector_size, opt.n_class]
img_predictors = [PredictNet(predict_param) for i in range(2)]
text_predictors = [PredictNet(predict_param) for i in range(2)]
faster_rcnn_lstm = FasterRCNN_LSTM(faster_extractor, lstm, img_predictors,text_predictors)

loading annotations into memory...
Done (t=1.97s)
creating index...
index created!
loading annotations into memory...
Done (t=24.69s)
creating index...
index created!
loading annotations into memory...
Done (t=0.15s)
creating index...
index created!
loading annotations into memory...
Done (t=0.82s)
creating index...
index created!


Exception in user code:
------------------------------------------------------------


Traceback (most recent call last):
  File "G:\Anaconda3\lib\site-packages\urllib3\connection.py", line 159, in _new_conn
    (self._dns_host, self.port), self.timeout, **extra_kw)
  File "G:\Anaconda3\lib\site-packages\urllib3\util\connection.py", line 80, in create_connection
    raise err
  File "G:\Anaconda3\lib\site-packages\urllib3\util\connection.py", line 70, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [WinError 10061] 由于目标计算机积极拒绝，无法连接。

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "G:\Anaconda3\lib\site-packages\urllib3\connectionpool.py", line 600, in urlopen
    chunked=chunked)
  File "G:\Anaconda3\lib\site-packages\urllib3\connectionpool.py", line 354, in _make_request
    conn.request(method, url, **httplib_request_kw)
  File "G:\Anaconda3\lib\http\client.py", line 1229, in request
    self._send_request(method, url, body, headers, encode_chunked)
  File "G:\Anaconda3\lib\http\client.py

In [3]:
data_iter = iter(dataloader)
image,text,bbox,labels = next(data_iter)

torch.Size([3, 424, 640])
torch.Size([3, 480, 640])
torch.Size([3, 640, 431])
torch.Size([3, 640, 427])
torch.Size([3, 480, 640])
torch.Size([3, 427, 640])
torch.Size([3, 487, 500])
torch.Size([3, 428, 640])
torch.Size([3, 435, 640])
torch.Size([3, 480, 640])
torch.Size([3, 424, 640])
torch.Size([3, 480, 640])
torch.Size([3, 612, 612])
torch.Size([3, 399, 640])
torch.Size([3, 480, 640])
torch.Size([3, 360, 640])
torch.Size([3, 458, 640])
torch.Size([3, 640, 480])
torch.Size([3, 480, 640])
torch.Size([3, 640, 479])
torch.Size([3, 333, 500])
torch.Size([3, 480, 640])
torch.Size([3, 480, 640])


In [ ]:
image[0][0].shape

torch.Size([1, 3, 424, 640])

In [ ]:
faster_rcnn_lstm.train_step(image, text, bbox, labels,opt.m,opt.k)

In [ ]:
for epoch in range(opt.epoch):
    for ii, (image,text,bbox,labels) in tqdm(enumerate(dataloader)):
        faster_rcnn_lstm.train_step(image, text, bbox, labels,opt.m,opt.k)